# Update map information from repository

A simple R script to read from the configuration file in json format and update information in database regarding valid maps.


## Set-up
Load libraries

In [1]:
require("RPostgreSQL")
require(jsonlite)
drv <- dbDriver("PostgreSQL") ## remember to update .pgpass file

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: jsonlite



Connect to the database using information in programming environment:

In [2]:
con <- dbConnect(drv, dbname = Sys.getenv("DBNAME"),
              host = Sys.getenv("DBHOST"),
              port = Sys.getenv("DBPORT"),
              user = Sys.getenv("DBUSER"))

## Read information on layers used in the webapp

First read json file

In [3]:
config.json <- read_json(sprintf("%s/data/config/groups.json", Sys.getenv("WEBDATAREPO")))


Loop through the items in the `json` file, extract the layer component, prepare and execute the `INSERT` query.

In [4]:
qrystr <- "INSERT INTO map_files(map_code,map_version,map_file,file_description,file_comments) 
values('%s','%s','%s','%s','%s') ON CONFLICT DO NOTHING"

for (j in config.json) {
    if ("layer" %in% names(j)) {
        if (j$layer$source == 'github') {
            layername <- strsplit(j$layer$path,"/")[[1]][2]
            mapinfo <- strsplit(layername,"_",fixed = 3)[[1]]
            qry = sprintf(qrystr,mapinfo[1],mapinfo[2],j$layer$path,j$layer$source,j$layer$type)
            #print(qry)
            dbSendQuery(con,qry)
            
        } else {
            ## print(j$id)
            ## print(j$layer$tileset)
        }
    }
}

In [5]:
print(qry)

[1] "INSERT INTO map_files(map_code,map_version,map_file,file_description,file_comments) \nvalues('TF1.7.web.orig','v1.0','layers/TF1.7.web.orig_v1.0_topo.json','github','topojson') ON CONFLICT DO NOTHING"


Disconnect from database

In [6]:
dbDisconnect(con)

[1] TRUE